ЦЕЛЬ
 Исследовать поведение пользователей в обновлённом приложении.

 ЗАДАЧИ
1. Зависит ли вероятность оплаты от выбранного пользователем уровня сложности
2. Существует ли разница во времени между событиями регистрации и оплаты для разных групп пользователей с разным уровнем сложности

In [170]:
# Загружаем данные
import pandas as pd
events_df = pd.read_csv('DA2021/7_4_Events.csv')
events = events_df.copy() # создаем копию исходных данных
cond = (events.start_time>='2018-01-01') & (events.start_time<'2019-01-01') & (events.event_type=='registration') #фильтруем данные за 2018 год
registered = events[cond]['user_id'].to_list()
events = events[events.user_id.isin(registered)]
events.start_time = pd.to_datetime(events.start_time, format='%Y-%m-%dT%H:%M:%S') # преобразуем формат даты


purchase_df = pd.read_csv('DA2021/purchase.csv')
purchase = purchase_df.copy() # создаем копию исходных данных
purchase['event_type'] = 'purchase' # создаем создаем столбец , готовим данные для дальнейшего объединения
purchase1 = purchase[purchase.user_id.isin(registered)]

#Проводим анализ данных

events[events['event_type'] == 'level_choice'].info()
events[events['event_type'].isin(['tutorial_start','tutorial_finish'])].info()
display(events.info())
events.describe(include='all') #оцениваем характеристики каждого столбца.
events['event_type'].unique() #оцениваем, какие уникальные события есть в колонках event_type и selected_level.
events['selected_level'].unique() 
events['user_id'].nunique() #какое количество пользователей совершали события

purchase1.head(10) #как события, совершённые пользователями, связаны с последующими оплатами пакетов вопросов
display(purchase1.info())
purchase1.describe()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8342 entries, 51424 to 118363
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   id              8342 non-null   int64         
 1   event_type      8342 non-null   object        
 2   selected_level  8342 non-null   object        
 3   start_time      8342 non-null   datetime64[ns]
 4   tutorial_id     0 non-null      float64       
 5   user_id         8342 non-null   int64         
dtypes: datetime64[ns](1), float64(1), int64(2), object(2)
memory usage: 456.2+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 32954 entries, 51412 to 118360
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   id              32954 non-null  int64         
 1   event_type      32954 non-null  object        
 2   selected_level  0 non-null      object        
 3   start_time 

None

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1600 entries, 1171 to 2778
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              1600 non-null   int64 
 1   user_id         1600 non-null   int64 
 2   event_datetime  1600 non-null   object
 3   amount          1600 non-null   int64 
 4   event_type      1600 non-null   object
dtypes: int64(3), object(2)
memory usage: 75.0+ KB


<ipython-input-170-6d82c2562136>:21: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  events.describe(include='all') #оцениваем характеристики каждого столбца.


None

,id,user_id,amount
count,1600.000000,1600.000000,1600.000000
mean,17645.505625,37752.766250,110.734375
std,462.038637,5822.621784,54.696628
min,16845.000000,27845.000000,25.000000
25%,17245.750000,32815.750000,50.000000
50%,17645.500000,37633.500000,100.000000
75%,18045.250000,43023.000000,150.000000
max,18452.000000,47742.000000,300.000000


In [171]:
# Исследование поведения пользователей в приложении
#1. Получим данные из файлов
import pandas as pd
events_df = pd.read_csv('DA2021/7_4_Events.csv')
events = events_df.copy()
purchase_df = pd.read_csv('DA2021/purchase.csv')
purchase = purchase_df.copy()
display(purchase)
display(events)

,id,user_id,event_datetime,amount
0,15674,12584,2016-05-12T10:34:16,100
1,15675,12985,2016-05-13T08:25:56,50
2,15676,12828,2016-05-13T16:33:46,50
3,15677,12598,2016-05-14T01:09:37,150
4,15678,13037,2016-05-14T01:24:46,100
...,...,...,...,...
5951,21625,87331,2020-07-06T09:02:07,50
5952,21626,87418,2020-07-06T14:16:37,100
5953,21627,87431,2020-07-06T22:48:59,50
5954,21628,87363,2020-07-07T05:38:56,100


,id,event_type,selected_level,start_time,tutorial_id,user_id
0,28903,registration,NaN,2016-05-11T23:40:55,NaN,12583
1,28904,registration,NaN,2016-05-11T23:49:58,NaN,12584
2,28905,registration,NaN,2016-05-12T00:53:07,NaN,12585
3,28906,tutorial_start,NaN,2016-05-12T01:32:20,17562.0,12585
4,28907,tutorial_finish,NaN,2016-05-12T01:34:53,17562.0,12585
...,...,...,...,...,...,...
252329,281232,level_choice,hard,2020-07-02T10:02:15,NaN,87439
252330,281233,level_choice,medium,2020-07-02T11:38:52,NaN,87488
252331,281234,pack_choice,NaN,2020-07-02T11:42:14,NaN,87488
252332,281235,tutorial_start,NaN,2020-07-02T13:32:58,86127.0,87464


In [172]:
# 2. Произведем обзор данных
display(events.info())
display(purchase.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252334 entries, 0 to 252333
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   id              252334 non-null  int64  
 1   event_type      252334 non-null  object 
 2   selected_level  31086 non-null   object 
 3   start_time      252334 non-null  object 
 4   tutorial_id     125103 non-null  float64
 5   user_id         252334 non-null  int64  
dtypes: float64(1), int64(2), object(3)
memory usage: 11.6+ MB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5956 entries, 0 to 5955
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              5956 non-null   int64 
 1   user_id         5956 non-null   int64 
 2   event_datetime  5956 non-null   object
 3   amount          5956 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 186.2+ KB


None

In [173]:
# Преобразуем данные
cond = (events.start_time>='2018-01-01') & (events.start_time<'2019-01-01') & (events.event_type=='registration')
registered = events[cond]['user_id'].to_list()
events = events[events.user_id.isin(registered)]
events.start_time = pd.to_datetime(events.start_time, format='%Y-%m-%dT%H:%M:%S')
display(events)

purchase.event_datetime = pd.to_datetime(purchase.event_datetime, format='%Y-%m-%dT%H:%M:%S')
#Добавим в датафрейм purchase столбец event_type, который будет содержать одно значение purchase. 
# Это нужно, чтобы в объединённом датафрейме однозначно выделить события оплаты.
purchase['event_type'] = 'purchase'
purchase1 = purchase[purchase.user_id.isin(registered)]

display(purchase1)

,id,event_type,selected_level,start_time,tutorial_id,user_id
51405,80308,registration,NaN,2018-01-01 03:48:40,NaN,27832
51406,80309,registration,NaN,2018-01-01 04:07:25,NaN,27833
51407,80310,registration,NaN,2018-01-01 08:35:10,NaN,27834
51408,80311,registration,NaN,2018-01-01 11:54:47,NaN,27835
51409,80312,registration,NaN,2018-01-01 13:28:07,NaN,27836
...,...,...,...,...,...,...
118359,147262,tutorial_start,NaN,2019-01-01 03:26:18,49554.0,47756
118360,147263,tutorial_finish,NaN,2019-01-01 03:29:25,49554.0,47756
118362,147265,level_choice,easy,2019-01-01 05:04:52,NaN,47755
118363,147266,level_choice,medium,2019-01-01 05:42:11,NaN,47756


,id,user_id,event_datetime,amount,event_type
1171,16845,27845,2018-01-03 18:53:43,100,purchase
1172,16846,27865,2018-01-04 14:46:10,250,purchase
1174,16848,27911,2018-01-07 08:19:12,50,purchase
1175,16849,27910,2018-01-07 12:11:34,100,purchase
1176,16850,27940,2018-01-07 13:16:41,200,purchase
...,...,...,...,...,...
2767,18441,47498,2019-01-02 03:48:19,100,purchase
2768,18442,47647,2019-01-02 23:26:26,150,purchase
2769,18443,47554,2019-01-03 00:36:36,50,purchase
2774,18448,47742,2019-01-04 12:51:41,50,purchase


In [174]:
# Преобразуем данные
events = events.rename(columns={"id": "event_id"}) #переименуем столбцы с одинаковым названием
purchase1 = purchase1.rename(columns={"id": "purchase_id"})

In [175]:
# Определяем процент пользователей, выбравших уровень сложности от общего числа зарегистрированных пользователей
registered_users_count = events[events["event_type"] == "registration"][
    "user_id"
].nunique()
level_choice_users_count = events[events["event_type"] == "level_choice"][
    "user_id"
].nunique()
percent_level_choice_users = level_choice_users_count / registered_users_count
print(
    "Процент пользователей, выбравших уровень сложности (от общего числа зарегистрировавшихся): {:.2%}".format(
        percent_level_choice_users
    )
)
#Меньше половины пользователей (41.86 %) доходят до этапа выбора уровня сложности вопросов. 
# Этот этап напрямую влияет на то, что пользователь будет пользоваться приложением через бесплатные возможности, которые в дальнейшем могут привести к оплате.

Процент пользователей, выбравших уровень сложности (от общего числа зарегистрировавшихся): 41.86%


In [176]:
# Определяем процент пользователей, которые оплатили вопросы (от числа пользователей, которые выбрали уровень сложности
paying_users_count = purchase1["user_id"].nunique()
percent_of_paying_users = paying_users_count / level_choice_users_count
print(
    "Процент пользователей, которые оплатили вопросы (от числа пользователей, которые выбрали уровень сложности): {:.2%}".format(
        percent_of_paying_users
    )
)
print(paying_users_count )
print(level_choice_users_count )
# Процент пользователей осуществивших оплату очень низкий и составляет 19.18%
# Количество пользователей совершивших оплату 1600
# Количество пользователей совершивших действие выбора цровня сложности 8342

Процент пользователей, которые оплатили вопросы (от числа пользователей, которые выбрали уровень сложности): 19.18%
1600
8342


In [177]:
#Определим перечень уровней сложности, которые выбирают пользователи
b = events.groupby('selected_level')['user_id'].count()
print(b)
# Пользователи выбирают три уровня сложности easy(легкий), medium(средний), hard (тяжелый). Большая часть пользователей (около 50 %) выбирают средний уровень сложности.

selected_level
easy      2448
hard      1249
medium    4645
Name: user_id, dtype: int64


In [238]:
#Формируем датафраймы по уровням сложности
easy_users_count = events[events['selected_level'] == 'easy']
medium_users_count = events[events['selected_level'] == 'medium']
hard_users_count = events[events['selected_level'] == 'hard']
display(easy_users_count)

,event_id,event_type,selected_level,start_time,tutorial_id,user_id
51450,80353,level_choice,easy,2018-01-02 08:46:03,NaN,27842
51457,80360,level_choice,easy,2018-01-02 11:53:11,NaN,27849
51493,80396,level_choice,easy,2018-01-03 00:39:54,NaN,27853
51497,80400,level_choice,easy,2018-01-03 01:11:45,NaN,27859
51513,80416,level_choice,easy,2018-01-03 09:31:00,NaN,27863
...,...,...,...,...,...,...
118267,147170,level_choice,easy,2018-12-30 12:28:11,NaN,47725
118297,147200,level_choice,easy,2018-12-30 19:44:52,NaN,47733
118317,147220,level_choice,easy,2018-12-31 02:23:50,NaN,47743
118335,147238,level_choice,easy,2018-12-31 07:53:04,NaN,47744


In [244]:
# Объединяем таблицы датафрамов легкого уровня с таблицей данных оплаты
easy_users_count_df = pd.concat([easy_users_count,purchase1],sort=False)
# Теперь сбросим индексы объединённого датафрейма (так как после объединения они дублировались и несут мало смысла) с помощью метода reset_index() 
# и отсортируем все события по возрастанию времени с помощью sort_values().
easy_users_count_df = easy_users_count_df.reset_index(drop=True).sort_values('start_time')
easy_users_count_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4048 entries, 0 to 4047
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   event_id        2448 non-null   float64       
 1   event_type      4048 non-null   object        
 2   selected_level  2448 non-null   object        
 3   start_time      2448 non-null   datetime64[ns]
 4   tutorial_id     0 non-null      float64       
 5   user_id         4048 non-null   int64         
 6   purchase_id     1600 non-null   float64       
 7   event_datetime  1600 non-null   datetime64[ns]
 8   amount          1600 non-null   float64       
dtypes: datetime64[ns](2), float64(4), int64(1), object(2)
memory usage: 316.2+ KB


In [259]:
# Объединяем таблицы датафрамов среднего уровня с таблицей данных оплаты
medium_users_count_df = pd.concat([medium_users_count,purchase1],sort=False)
# Теперь сбросим индексы объединённого датафрейма (так как после объединения они дублировались и несут мало смысла) с помощью метода reset_index() 
# и отсортируем все события по возрастанию времени с помощью sort_values().
medium_users_count_df = medium_users_count_df.reset_index(drop=True).sort_values('start_time')
medium_users_count_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6245 entries, 0 to 6244
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   event_id        4645 non-null   float64       
 1   event_type      6245 non-null   object        
 2   selected_level  4645 non-null   object        
 3   start_time      4645 non-null   datetime64[ns]
 4   tutorial_id     0 non-null      float64       
 5   user_id         6245 non-null   int64         
 6   purchase_id     1600 non-null   float64       
 7   event_datetime  1600 non-null   datetime64[ns]
 8   amount          1600 non-null   float64       
dtypes: datetime64[ns](2), float64(4), int64(1), object(2)
memory usage: 487.9+ KB


In [242]:
# Объединяем таблицы датафрамов тяжелого уровня с таблицей данных оплаты
hard_users_count_df = pd.concat([hard_users_count,purchase1],sort=False)
# Теперь сбросим индексы объединённого датафрейма (так как после объединения они дублировались и несут мало смысла) с помощью метода reset_index() 
# и отсортируем все события по возрастанию времени с помощью sort_values().
hard_users_count_df = hard_users_count_df.reset_index(drop=True).sort_values('start_time')
hard_users_count_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2849 entries, 0 to 2848
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   event_id        1249 non-null   float64       
 1   event_type      2849 non-null   object        
 2   selected_level  1249 non-null   object        
 3   start_time      1249 non-null   datetime64[ns]
 4   tutorial_id     0 non-null      float64       
 5   user_id         2849 non-null   int64         
 6   purchase_id     1600 non-null   float64       
 7   event_datetime  1600 non-null   datetime64[ns]
 8   amount          1600 non-null   float64       
dtypes: datetime64[ns](2), float64(4), int64(1), object(2)
memory usage: 222.6+ KB


In [261]:
# Определяем среднее кол-во действий, которое пользователей совершившил, а затем выводи важные параметры, по датафрейму со средним уровнем сложности
medium_purchase_df = medium_users_count_df[medium_users_count_df["event_type"] == "purchase"]
print(medium_purchase_df["user_id"].value_counts().mean())
medium_purchase_df = medium_purchase_df[["user_id", "event_datetime"]].rename(
    columns={"event_datetime": "purchase_time"}
)

medium_level_choice_df = medium_users_count_df[medium_users_count_df["event_type"] == "level_choice"]
print(medium_level_choice_df["user_id"].value_counts().mean())
medium_level_choice_df = medium_level_choice_df[["user_id", "start_time"]].rename(
    columns={"start_time": "level_choice_time"}
)

merged_df_6 = medium_level_choice_df.merge(medium_purchase_df, on="user_id", how="inner")

merged_df_6["timedelta"] = (merged_df_6["purchase_time"] - merged_df_6["level_choice_time"])
print(merged_df_6["timedelta"].mean())
print(merged_df_6["timedelta"].describe())
# Количество оплативших пользователей 969, среднее время между действиями составляет 3 дня 23 часов 14 минут 13 секунды

1.0
1.0
3 days 23:14:13.165118679
count                          969
mean     3 days 23:14:13.165118679
std      2 days 06:18:57.618467109
min                0 days 04:18:12
25%                2 days 01:20:07
50%                3 days 19:53:19
75%                5 days 16:07:19
max               10 days 13:51:01
Name: timedelta, dtype: object


In [282]:
# Находим процен пользователей выбравших уровень сложности средний и выполнивших оплату.
medium_purchase_df = 969/4645
print(
    "Процент пользователей, которые оплатили вопросы (от числа пользователей, которые выбрали уровень сложности): {:.2%}".format(
        medium_purchase_df
    )
)
# Процент пользователей, которые оплатили вопросы (от числа пользователей, которые выбрали уровень сложности): 20,86%

Процент пользователей, которые оплатили вопросы (от числа пользователей, которые выбрали уровень сложности): 20.86%


In [249]:
# Определяем среднее кол-во действий, которое пользователей совершившил, а затем выводи важные параметры, по датафрейму с легким уровнем сложности
easy_purchase_df = easy_users_count_df[easy_users_count_df["event_type"] == "purchase"]
print(easy_purchase_df["user_id"].value_counts().mean())
easy_purchase_df = easy_purchase_df[["user_id", "event_datetime"]].rename(
    columns={"event_datetime": "purchase_time"}
)

easy_level_choice_df = easy_users_count_df[easy_users_count_df["event_type"] == "level_choice"]
print(easy_level_choice_df["user_id"].value_counts().mean())
easy_level_choice_df = easy_level_choice_df[["user_id", "start_time"]].rename(
    columns={"start_time": "level_choice_time"}
)

merged_df_6 = easy_level_choice_df.merge(easy_purchase_df, on="user_id", how="inner")
merged_df_6["timedelta"] = (merged_df_6["purchase_time"] - merged_df_6["level_choice_time"])
print(merged_df_6["timedelta"].mean())
print(merged_df_6["timedelta"].describe())
# Количество оплативших пользователей 189, среднее время между действиями составляет 3 дня 14 часов 58 минут 52 секунды

1.0
1.0
3 days 14:58:52.941798941
count                          189
mean     3 days 14:58:52.941798941
std      2 days 07:06:35.644097504
min                0 days 00:49:20
25%                1 days 17:18:56
50%                3 days 06:03:50
75%                5 days 06:58:18
max               10 days 18:35:09
Name: timedelta, dtype: object


In [283]:
# Находим процен пользователей выбравших уровень сложности легкий и выполнивших оплату.
easy_purchase_df = 189/2448
print(
    "Процент пользователей, которые оплатили вопросы (от числа пользователей, которые выбрали уровень сложности): {:.2%}".format(
        easy_purchase_df
    )
)
# Процент пользователей, которые оплатили вопросы (от числа пользователей, которые выбрали уровень сложности): 7,72%

Процент пользователей, которые оплатили вопросы (от числа пользователей, которые выбрали уровень сложности): 7.72%


In [251]:
# Определяем среднее кол-во действий, которое пользователей совершившил, а затем выводи важные параметры, по датафрейму с тяжелым уровнем сложности
hard_purchase_df = hard_users_count_df[hard_users_count_df["event_type"] == "purchase"]
print(hard_purchase_df["user_id"].value_counts().mean())
hard_purchase_df = hard_purchase_df[["user_id", "event_datetime"]].rename(
    columns={"event_datetime": "purchase_time"}
)

hard_level_choice_df = hard_users_count_df[hard_users_count_df["event_type"] == "level_choice"]
print(hard_level_choice_df["user_id"].value_counts().mean())
hard_level_choice_df = hard_level_choice_df[["user_id", "start_time"]].rename(
    columns={"start_time": "level_choice_time"}
)

merged_df_6 = hard_level_choice_df.merge(hard_purchase_df, on="user_id", how="inner")
merged_df_6["timedelta"] = (merged_df_6["purchase_time"] - merged_df_6["level_choice_time"])
print(merged_df_6["timedelta"].mean())
print(merged_df_6["timedelta"].describe())
# Количество оплативших пользователей 442, среднее время между действиями составляет 3 дня 7 часов 20 минут 41 секунда

1.0
1.0
3 days 07:20:41.420814479
count                          442
mean     3 days 07:20:41.420814479
std      1 days 21:43:52.953292605
min                0 days 03:26:45
25%         1 days 14:57:23.500000
50%         3 days 03:13:57.500000
75%         4 days 19:16:00.250000
max                8 days 01:18:13
Name: timedelta, dtype: object


In [281]:
# Находим процен пользователей выбравших уровень сложности тяжелый и выполнивших оплату.
hard_purchase_df = 442/1249
print(
    "Процент пользователей, которые оплатили вопросы (от числа пользователей, которые выбрали уровень сложности): {:.2%}".format(
        hard_purchase_df
    )
)
# Процент пользователей, которые оплатили вопросы (от числа пользователей, которые выбрали уровень сложности): 35,39%

Процент пользователей, которые оплатили вопросы (от числа пользователей, которые выбрали уровень сложности): 35.39%


In [262]:
# Объединяем таблицы датафрамов данных об операциях с таблицей данных оплаты
events_users_count_df = pd.concat([events,purchase1],sort=False)
#Теперь сбросим индексы объединённого датафрейма (так как после объединения они дублировались и несут мало смысла) с помощью метода reset_index() 
# и отсортируем все события по возрастанию времени с помощью sort_values().
events_users_count_df = events_users_count_df.reset_index(drop=True).sort_values('start_time')
events_users_count_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 68559 entries, 0 to 68558
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   event_id        66959 non-null  float64       
 1   event_type      68559 non-null  object        
 2   selected_level  8342 non-null   object        
 3   start_time      66959 non-null  datetime64[ns]
 4   tutorial_id     32954 non-null  float64       
 5   user_id         68559 non-null  int64         
 6   purchase_id     1600 non-null   float64       
 7   event_datetime  1600 non-null   datetime64[ns]
 8   amount          1600 non-null   float64       
dtypes: datetime64[ns](2), float64(4), int64(1), object(2)
memory usage: 5.2+ MB


In [264]:
# Определяем среднее кол-во действий, которое пользователей совершившил, а затем выводи важные параметры, по объединенному датафрейму
events_purchase_df = events_users_count_df[events_users_count_df["event_type"] == "purchase"]
print(events_purchase_df["user_id"].value_counts().mean())
events_purchase_df = events_purchase_df[["user_id", "event_datetime"]].rename(
    columns={"event_datetime": "purchase_time"}
)

events_level_choice_df = events_users_count_df[events_users_count_df["event_type"] == "level_choice"]
print(events_level_choice_df["user_id"].value_counts().mean())
events_level_choice_df = events_level_choice_df[["user_id", "start_time"]].rename(
    columns={"start_time": "level_choice_time"}
)

merged_df_6 = events_level_choice_df.merge(events_purchase_df, on="user_id", how="inner")
merged_df_6["timedelta"] = (merged_df_6["purchase_time"] - merged_df_6["level_choice_time"])
print(merged_df_6["timedelta"].mean())
print(merged_df_6["timedelta"].describe())
# Всего оплат 1600, срений интервал времени между выбором уровня и оплатой составляет 3 дня 17 часов 52 минуты 17 секунд

1.0
1.0
3 days 17:52:17.719375
count                         1600
mean        3 days 17:52:17.719375
std      2 days 04:37:14.656359486
min                0 days 00:49:20
25%         1 days 21:27:33.750000
50%                3 days 12:59:27
75%         5 days 09:47:33.750000
max               10 days 18:35:09
Name: timedelta, dtype: object


In [267]:
# Формируем датафрайм с данными по регистрации
registration_df = events[events['event_type'] == 'registration']
registration_df['user_id'].value_counts().mean()

# Объединяем таблицы данными регистрации и данными тяжелого уровня
registration_hard_users_count_df = pd.concat([hard_users_count_df,registration_df],sort=False)
#Теперь сбросим индексы объединённого датафрейма (так как после объединения они дублировались и несут мало смысла) с помощью метода reset_index() 
# и отсортируем все события по возрастанию времени с помощью sort_values().
registration_hard_users_count_df = registration_hard_users_count_df.reset_index(drop=True).sort_values('start_time')
registration_hard_users_count_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22775 entries, 2849 to 2848
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   event_id        21175 non-null  float64       
 1   event_type      22775 non-null  object        
 2   selected_level  1249 non-null   object        
 3   start_time      21175 non-null  datetime64[ns]
 4   tutorial_id     0 non-null      float64       
 5   user_id         22775 non-null  int64         
 6   purchase_id     1600 non-null   float64       
 7   event_datetime  1600 non-null   datetime64[ns]
 8   amount          1600 non-null   float64       
dtypes: datetime64[ns](2), float64(4), int64(1), object(2)
memory usage: 1.7+ MB


In [274]:
# Определяем среднее кол-во действий, которое пользователей совершившил, а затем выводи важные параметры, по объединенному датафрейму
events_purchase_df = registration_hard_users_count_df[registration_hard_users_count_df["event_type"] == "level_choice"]
print(events_purchase_df["user_id"].value_counts().mean())
events_purchase_df = events_purchase_df[["user_id", "start_time"]].rename(
    columns={"start_time": "purchase_time"}
)

events_level_choice_df = registration_hard_users_count_df[registration_hard_users_count_df["event_type"] == "registration"]
print(events_level_choice_df["user_id"].value_counts().mean())
events_level_choice_df = events_level_choice_df[["user_id", "start_time"]].rename(
    columns={"start_time": "level_choice_time"}
)

merged_df_6 = events_level_choice_df.merge(events_purchase_df, on="user_id", how="inner")
merged_df_6["timedelta"] = (merged_df_6["purchase_time"] - merged_df_6["level_choice_time"])
print(merged_df_6["timedelta"].mean())
print(merged_df_6["timedelta"].describe())
# Средний интервал времени между регистрацией и выбором уровня сложности тяжелый составляет 7 часов 18 минуты 32 секунды

1.0
1.0
0 days 07:18:32.228182546
count                         1249
mean     0 days 07:18:32.228182546
std      0 days 04:36:04.422579172
min                0 days 00:08:59
25%                0 days 03:58:18
50%                0 days 06:13:02
75%                0 days 10:03:27
max                1 days 06:43:55
Name: timedelta, dtype: object


In [275]:
# Объединяем таблицы данными регистрации и данными среднего уровня
registration_medium_users_count_df = pd.concat([medium_users_count_df,registration_df],sort=False)
#Теперь сбросим индексы объединённого датафрейма (так как после объединения они дублировались и несут мало смысла) с помощью метода reset_index() 
# и отсортируем все события по возрастанию времени с помощью sort_values().
registration_medium_users_count_df = registration_medium_users_count_df.reset_index(drop=True).sort_values('start_time')
registration_medium_users_count_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26171 entries, 6245 to 6244
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   event_id        24571 non-null  float64       
 1   event_type      26171 non-null  object        
 2   selected_level  4645 non-null   object        
 3   start_time      24571 non-null  datetime64[ns]
 4   tutorial_id     0 non-null      float64       
 5   user_id         26171 non-null  int64         
 6   purchase_id     1600 non-null   float64       
 7   event_datetime  1600 non-null   datetime64[ns]
 8   amount          1600 non-null   float64       
dtypes: datetime64[ns](2), float64(4), int64(1), object(2)
memory usage: 2.0+ MB


In [276]:
# Определяем среднее кол-во действий, которое пользователей совершившил, а затем выводи важные параметры, по объединенному датафрейму
events_purchase_df = registration_medium_users_count_df[registration_medium_users_count_df["event_type"] == "level_choice"]
print(events_purchase_df["user_id"].value_counts().mean())
events_purchase_df = events_purchase_df[["user_id", "start_time"]].rename(
    columns={"start_time": "purchase_time"}
)

events_level_choice_df = registration_medium_users_count_df[registration_medium_users_count_df["event_type"] == "registration"]
print(events_level_choice_df["user_id"].value_counts().mean())
events_level_choice_df = events_level_choice_df[["user_id", "start_time"]].rename(
    columns={"start_time": "level_choice_time"}
)

merged_df_6 = events_level_choice_df.merge(events_purchase_df, on="user_id", how="inner")
merged_df_6["timedelta"] = (merged_df_6["purchase_time"] - merged_df_6["level_choice_time"])
print(merged_df_6["timedelta"].mean())
print(merged_df_6["timedelta"].describe())
# Средний интервал времени между регистрацией и выбором уровня сложности средний составляет 7 часов 08 минуты 13 секунды

1.0
1.0
0 days 07:08:13.360387513
count                         4645
mean     0 days 07:08:13.360387513
std      0 days 04:31:51.219231339
min                0 days 00:08:15
25%                0 days 03:50:32
50%                0 days 06:01:39
75%                0 days 09:32:00
max                1 days 07:02:41
Name: timedelta, dtype: object


In [279]:
# Объединяем таблицы данными регистрации и данными среднего уровня
registration_easy_users_count_df = pd.concat([easy_users_count_df,registration_df],sort=False)
#Теперь сбросим индексы объединённого датафрейма (так как после объединения они дублировались и несут мало смысла) с помощью метода reset_index() 
# и отсортируем все события по возрастанию времени с помощью sort_values().
registration_easy_users_count_df = registration_easy_users_count_df.reset_index(drop=True).sort_values('start_time')
registration_easy_users_count_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23974 entries, 4048 to 4047
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   event_id        22374 non-null  float64       
 1   event_type      23974 non-null  object        
 2   selected_level  2448 non-null   object        
 3   start_time      22374 non-null  datetime64[ns]
 4   tutorial_id     0 non-null      float64       
 5   user_id         23974 non-null  int64         
 6   purchase_id     1600 non-null   float64       
 7   event_datetime  1600 non-null   datetime64[ns]
 8   amount          1600 non-null   float64       
dtypes: datetime64[ns](2), float64(4), int64(1), object(2)
memory usage: 1.8+ MB


In [280]:
# Определяем среднее кол-во действий, которое пользователей совершившил, а затем выводи важные параметры, по объединенному датафрейму
events_purchase_df = registration_easy_users_count_df[registration_easy_users_count_df["event_type"] == "level_choice"]
print(events_purchase_df["user_id"].value_counts().mean())
events_purchase_df = events_purchase_df[["user_id", "start_time"]].rename(
    columns={"start_time": "purchase_time"}
)

events_level_choice_df = registration_easy_users_count_df[registration_easy_users_count_df["event_type"] == "registration"]
print(events_level_choice_df["user_id"].value_counts().mean())
events_level_choice_df = events_level_choice_df[["user_id", "start_time"]].rename(
    columns={"start_time": "level_choice_time"}
)

merged_df_6 = events_level_choice_df.merge(events_purchase_df, on="user_id", how="inner")
merged_df_6["timedelta"] = (merged_df_6["purchase_time"] - merged_df_6["level_choice_time"])
print(merged_df_6["timedelta"].mean())
print(merged_df_6["timedelta"].describe())
# Средний интервал времени между регистрацией и выбором уровня сложности легкий составляет 7 часов 10 минуты 6 секунды

1.0
1.0
0 days 07:10:06.324754901
count                         2448
mean     0 days 07:10:06.324754901
std      0 days 04:36:30.973389545
min                0 days 00:09:26
25%         0 days 03:55:05.750000
50%         0 days 06:02:43.500000
75%         0 days 09:23:38.500000
max                1 days 18:48:25
Name: timedelta, dtype: object


 Сведем данные по интервалам между выбором уровня сложности и оплатой:
Сремнее время между операциями  3 дня 17 часов 52 минуты 17 секунд
Тяжелый уровень среднее время между действиями составляет 3 дня 7 часов 20 минут 41 секунда, % оплат 35,39, количество пользователей 1249
Средний уровень среднее время между действиями составляет 3 дня 23 часов 14 минут 13 секунды, % оплат 20.86, количество пользователей 4645
Легкий уровень среднее время между действиями составляет 3 дня 14 часов 58 минут 52 секунды, % оплат 7,72, количество пользователей 2448
Из полученных данных видно, что для нас наиболее привлекательны пользователи , выбирающие средний и тяжелый уровень сложности. 
Наибольшее количество пользователей выбирают средний уровень, а так же они платят чаще. 
Тяжелый уровень выбирают продвинутые пользователи, уверенные в своих силах, но хотя их в 4 раза меньще, чем средних, они платят чаще.
Наиболее продолжительное время между действиями проходит у пользователей среднее уровня. Необходимо понять причину и оптимизировать это промежуток.

Сведем данные по интервалам между регистрацией и выбором уровня сложности:
Средний интервал времени между регистрацией и выбором уровня сложности легкий составляет 7 часов 10 минуты 6 секунды
Средний интервал времени между регистрацией и выбором уровня сложности средний составляет 7 часов 08 минуты 13 секунды
Средний интервал времени между регистрацией и выбором уровня сложности тяжелый составляет 7 часов 18 минуты 32 секунды
Из полученных данных видно, что интевалы различаются в читанные минуты и их продолжительность не может влиять на интервал между регистрацией и оплатой.